In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X = pd.read_csv('./predict-supercars-prices-2025/supercars_train.csv', index_col='id')
X_test = pd.read_csv('./predict-supercars-prices-2025/supercars_test.csv', index_col='id')
X.head()

,year,brand,color,carbon_fiber_body,engine_config,horsepower,torque,weight_kg,zero_to_60_s,top_speed_mph,...,has_warranty,last_service_date,service_history,non_original_parts,model,warranty_years,damage,damage_cost,damage_type,price
id,,,,,,,,,,,,,,,,,,,,,
wn8zA4ADUC,2023,McLaren,Silver,1,Hybrid,1045,794,1897,3.63,227,...,0,2024-08-15,authorized,0,600LT,0,1,83632.0,major,329510.72
m5EyycSRrS,2024,Aston Martin,Black,1,W16,879,510,2193,2.72,247,...,0,2025-03-29,none,1,Valhalla,0,0,NaN,NaN,509289.91
IOcII96Ua3,2022,Koenigsegg,White,1,V12,609,489,1277,3.16,236,...,0,2025-06-30,authorized,0,Jesko,0,1,51179.0,major,2164428.25
qlds6yyR3r,2024,Bugatti,Blue,0,V12,1044,1162,1733,2.15,248,...,1,2024-08-29,none,0,Chiron,1,0,NaN,NaN,2793655.40
fRR6ulIWvU,2024,Pagani,White,0,V12,853,679,1307,2.35,247,...,0,2025-02-21,authorized,0,Zonda,0,1,77100.0,major,2927390.00


In [3]:
X_test.head()
X_test.columns

Index(['year', 'brand', 'color', 'carbon_fiber_body', 'engine_config',
       'horsepower', 'torque', 'weight_kg', 'zero_to_60_s', 'top_speed_mph',
       'num_doors', 'transmission', 'drivetrain', 'market_region', 'mileage',
       'num_owners', 'interior_material', 'brake_type', 'tire_brand',
       'aero_package', 'limited_edition', 'has_warranty', 'last_service_date',
       'service_history', 'non_original_parts', 'model', 'warranty_years',
       'damage', 'damage_cost', 'damage_type'],
      dtype='object')

In [4]:
X.dropna(axis= 0 , inplace=True, subset=['price'])
y = X['price']
X.drop(['price'],axis=1, inplace=True)

In [5]:
X_train_full,X_test_full,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
object_cols = [cols for cols in X_train_full.columns
               if X_train_full[cols].nunique() < 10 and
               X_train_full[cols].dtype == "object"]
numeric_cols = [cols for cols in X_train_full.columns
                if X_train_full[cols].dtype in ['int64', 'float64']]

print("object columns is a ", object_cols,"\n",len(object_cols))
print("numerice cols is a :", numeric_cols,"\n", len(numeric_cols))

object columns is a  ['brand', 'color', 'engine_config', 'transmission', 'drivetrain', 'market_region', 'interior_material', 'brake_type', 'tire_brand', 'service_history', 'damage_type'] 
 11
numerice cols is a : ['year', 'carbon_fiber_body', 'horsepower', 'torque', 'weight_kg', 'zero_to_60_s', 'top_speed_mph', 'num_doors', 'mileage', 'num_owners', 'aero_package', 'limited_edition', 'has_warranty', 'non_original_parts', 'warranty_years', 'damage', 'damage_cost'] 
 17


In [6]:
my_cols = object_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_test_full[my_cols].copy()
X_test = X_test[my_cols].copy()

X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)

X_train, X_valid = X_train.align(X_valid, join = 'left',axis=1)
X_train,X_test = X_train.align(X_test, join = 'left', axis=1)

In [7]:
from xgboost import XGBRegressor
model = XGBRegressor(random_state = 0)
model.fit(X_train,y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [8]:
prediction = model.predict(X_valid)

In [9]:
from sklearn.metrics import mean_absolute_error
mae1 = mean_absolute_error(y_test, prediction)
print("MAE:", mae1)

MAE: 332814.26138281246


In [10]:
my_model2 =XGBRegressor(n_estimators= 100, learning_rate = 0.05)
my_model2.fit(X_train,y_train)
pred2 = my_model2.predict(X_valid)
mae2 = mean_absolute_error(y_test, pred2)
print("MAE with tuned model:", mae2)


MAE with tuned model: 312812.73741875


In [38]:
my_model3 = XGBRegressor(n_estimators= 1000 ,learning_rate = 1, max_depth= 3, subsample= 0.5)
my_model3.fit(X_train,y_train)
pred3 = my_model3.predict(X_valid)
mae3 = mean_absolute_error(y_test, pred3)
print("MAE with tuned model:", mae3)


MAE with tuned model: 739578.1519417481


In [39]:
my_model4 =XGBRegressor(n_estimators= 1 )
my_model4.fit(X_train,y_train)
pred4 = my_model4.predict(X_valid)
mae4 = mean_absolute_error(y_test, pred4)
print("MAE with tuned model:", mae4)


MAE with tuned model: 730628.4080875


In [57]:
my_model5 =XGBRegressor(n_estimators= 1000, learning_rate = 0.1, max_depth= 1)
my_model5.fit(X_train,y_train)
pred5 = my_model5.predict(X_valid)
mae5 = mean_absolute_error(y_test, pred5)
print("MAE with tuned model:", mae5)


MAE with tuned model: 306368.60469999997


In [58]:
preds = my_model5.predict(X_test)
submission = pd.DataFrame({'id': X_test.index, 'target': preds})
submission.to_csv('submission.csv', index=False)

In [ ]:
my_model6 =XGBRegressor(n_estimators= 2000, learning_rate = 0.01, max_depth= 1)
my_model6.fit(X_train,y_train)
pred5 = my_model5.predict(X_valid)
mae5 = mean_absolute_error(y_test, pred5)
print("MAE with tuned model:", mae5)

MAE with tuned model: 305810.789365625


In [76]:
my_model5 =XGBRegressor(n_estimators= 2000, learning_rate = 0.01, max_depth= 1)
my_model5.fit(X_train,y_train)
pred5 = my_model5.predict(X_valid)
mae5 = mean_absolute_error(y_test, pred5)
print("MAE with tuned model:", mae5)


MAE with tuned model: 305810.789365625


In [78]:
preds = my_model6.predict(X_test)
submission = pd.DataFrame({'id': X_test.index, 'target': preds})
submission.to_csv('submission.csv', index=False)

In [84]:
my_model5 =XGBRegressor(n_estimators= 4000, learning_rate = 0.01, max_depth= 1)
my_model5.fit(X_train,y_train)
pred5 = my_model5.predict(X_valid)
mae5 = mean_absolute_error(y_test, pred5)
print("MAE with tuned model:", mae5)


MAE with tuned model: 304179.534846875


In [101]:
my_model7 =XGBRegressor(n_estimators= 4000, learning_rate = 0.01, max_depth= 1)
my_model7.fit(X_train,y_train)
pred7 = my_model7.predict(X_valid)
mae7 = mean_absolute_error(y_test, pred7)
print("MAE with tuned model:", mae7)

MAE with tuned model: 304179.534846875


In [102]:
preds = my_model7.predict(X_test)
submission = pd.DataFrame({'id': X_test.index, 'target': preds})
submission.to_csv('submission.csv', index=False)

In [11]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [500, 1000, 2000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=0),
    param_distributions=param_grid,
    n_iter=20,
    scoring='neg_mean_absolute_error',
    cv=5,
    verbose=1,
    n_jobs=-1
)
search.fit(X_train, y_train)
print("Best params:", search.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best params: {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 1.0}


In [14]:
model9 = XGBRegressor(n_estimators=500, learning_rate=0.01, max_depth=5, subsample=0.8, min_child_weight=1, colsample_bytree = 1.0)
model9.fit(X_train, y_train,
          eval_set=[(X_valid, y_test)],
          verbose=False)
pred9 = model9.predict(X_valid)
mae9 = mean_absolute_error(y_test, pred9)
print("MAE with tuned model:", mae9)


MAE with tuned model: 310128.39301875


In [18]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# Model
lgb_model = LGBMRegressor(
    n_estimators=5000,
    learning_rate=0.01,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=0
)

# Train with early stopping
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_test)],
    eval_metric='mae',
    
)

# Predict
preds_lgb = lgb_model.predict(X_valid)
mae_lgb = mean_absolute_error(y_test, preds_lgb)
print("LightGBM MAE:", mae_lgb)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001337 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1571
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 64
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 1375071.539053
LightGBM MAE: 329060.12898503826


In [19]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error

# Identify categorical features
cat_features = [i for i, col in enumerate(X_train.columns) if X_train[col].dtype == "object"]

# Model
cat_model = CatBoostRegressor(
    iterations=5000,
    learning_rate=0.05,
    depth=8,
    random_seed=0,
    loss_function='MAE',
    verbose=200
)

# Train
cat_model.fit(
    X_train, y_train,
    eval_set=(X_valid, y_test),
    cat_features=cat_features,
    early_stopping_rounds=50
)

# Predict
preds_cat = cat_model.predict(X_valid)
mae_cat = mean_absolute_error(y_test, preds_cat)
print("CatBoost MAE:", mae_cat)

0:	learn: 890117.9942404	test: 954758.2162776	best: 954758.2162776 (0)	total: 166ms	remaining: 13m 51s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 316239.2313
bestIteration = 114

Shrink model to first 115 iterations.
CatBoost MAE: 316239.2303917157
